# Retraining an AutoPredictor

Helps advanced users start with Amazon Forecast Retraining feature for an AutoPredictor

Predictors can be retained with updated datasets to keep your predictors up to date. When retraining a predictor, Amazon Forecast maintains the same predictor configuration settings. After retraining, the original predictor will remain active and the retrained predictor will have a separate Predictor ARN.

Retraining a predictor can improve forecasting accuracy in two ways:

1. *More current data*: Your retrained predictor will incorporate more up-to-date data when training a model.
2. *Predictor improvements*: Your retrained predictor will incorporate any updates and improvements in the Amazon Forecast algorithms and additional datasets.

Retraining a predictor can be up to 50% faster than creating a new predictor from scratch. Predictor training times are faster and AWS Forecast automatically uses your existing Predictor configuration during retraining to maintain model consistency.

Retraining a Predictor is different from creating a predictor from scratch and is expected to carry over your existing Predictor settings and train the underlying models on top of the latest data.

Note that you do not need to retrain everytime you are bringing in the new data and you can still generate forecast using the existing models. Retraining is required to periodically update the underlying models learning the latest data charecteristics and trends.

For more informations about APIs, refer API [documentation](https://docs.aws.amazon.com/forecast/latest/dg/API_Operations_Amazon_Forecast_Service.html)

For details about retraining a predictor, refer https://docs.aws.amazon.com/forecast/latest/dg/retrain-predictors.html

As part of this notebook, we will first create an auto predictor, import new data and then retrain the predictor so that we can get a new predictor trained with the latest available data.

## Table Of Contents
* [Pre-requisites](#prerequisites)
* [Test Setup - Creating a new AutoPredictor](#createAuto)
  * [Importing Data in Forecast System](#importingData)
  * [Creating the first Predictor](#firstPredictor)
* [Retraining an AutoPredictor](#retrainAuto)
* [Cleanup](#cleanup)

**Read Every Cell FULLY before executing it**


## Pre-requisites <a class="anchor" id="prerequisites"></a>
Before we get started, lets set up the notebook environment, the AWS SDK client for Amazon Forecast and IAM Role used by Amazon Forecast to access your data.

In [ ]:
import sys
import os
import time
import boto3

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

Configure the S3 bucket name and region name for this lesson.

- If you don't have an S3 bucket, create it first on S3.
- Although we have set the region to us-west-2 as a default value below, you can choose any of the regions that the service is available in.

In [ ]:
text_widget_bucket = util.create_text_widget( "bucketName", "input your S3 bucket name" )
text_widget_region = util.create_text_widget( "region", "input region name.", default_value="us-west-2" )

In [ ]:
bucketName = text_widget_bucket.value
assert bucketName, "bucket_name not set. Please set the bucket name and retry"

region = text_widget_region.value
assert region, "region not set."

In [ ]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Test Setup - Creating a new AutoPredictor<a class="anchor" id="createAuto"></a>


In Amazon Forecast , a dataset is a collection of file(s) which contain data that is relevant for a forecasting task. A dataset must conform to a schema provided by Amazon Forecast. 

For this exercise, we use the individual household electric power consumption dataset. (Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.) We aggregate the usage data hourly. 

As part of this notebook, we will be first importing subset of this data to create the first predictor and then use another subset(With additional data) to retrain the predictor 

### Importing Data in Forecast System<a class="anchor" id="importingData"></a>

This notebooks has two csv files in the "data" folder where the first file "elec_data_small_first.csv" will be used to create the first AutoPredictor and subsequent file will be used to bring in new data for retraining.

Amazon forecast can import data from Amazon S3. We first explore the data locally to see the fields

In [ ]:
import pandas as pd

df = pd.read_csv("data/elec_data_small_first.csv", dtype = object, header =None)
df.head()

Now upload the data to S3. But before doing that, go into your AWS Console, select S3 for the service and create a new bucket inside the `Oregon` or `us-west-2` region. Use that bucket name convention of `amazon-forecast-unique-value-data`. The name must be unique, if you get an error, just adjust until your name works, then update the `bucketName` cell below.

In [ ]:
s3 = session.client('s3')

In [ ]:
key_first_predictor="elec_data/item-demand-time_first.csv"

In [ ]:
s3.upload_file(Filename="data/elec_data_small_first.csv", Bucket=bucketName, Key=key_first_predictor)

In [ ]:
# Create the role to provide to Amazon Forecast.
role_name = "ForecastNotebookRole-Retrain"
role_arn = util.get_or_create_iam_role( role_name = role_name )

More details about `Domain` and dataset type can be found on the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/howitworks-domains-ds-types.html) . For this example, we are using [CUSTOM](https://docs.aws.amazon.com/forecast/latest/dg/custom-domain.html) domain with 3 required attributes `timestamp`, `target_value` and `item_id`. Also for your project name, update it to reflect your name in a lowercase format.

In [ ]:
DATASET_FREQUENCY = "H" 
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

In [ ]:
project = 'ws_retraining' # Replace this with a unique name here, make sure the entire name is < 30 characters.
datasetName= project+'_ds'
datasetGroupName= project +'_gp'
s3DataPath = "s3://"+bucketName+"/"+key_first_predictor

In [ ]:
print("DatasetName: ", datasetName)
print("DatasetGroupName: ", datasetGroupName)
print("S3 bucket: ", s3DataPath)

In [ ]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
                   )
datasetArn = response['DatasetArn']

In [ ]:
print("DatasetArn:", datasetArn)

In [ ]:
create_dataset_group_response = forecast.create_dataset_group(DatasetGroupName=datasetGroupName,
                                                              Domain="CUSTOM",
                                                              DatasetArns= [datasetArn]
                                                             )
datasetGroupArn = create_dataset_group_response['DatasetGroupArn']

Brings the data into Amazon Forecast system ready to forecast from raw data. 

In [ ]:
datasetImportJobName = 'Electricity_First_Import_TTS'
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [ ]:
ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print("DatasetImportJob", ds_import_job_arn)

Check the status of dataset, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on the data size. It can take 10 mins to be **ACTIVE**. This process will take 5 to 10 minutes.

In [ ]:
print(f"Waiting for Dataset Import Job with ARN {ds_import_job_arn} to become **ACTIVE**. This process could take 15-20 minutes.\n\nCurrent Status:")
status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn))

describe_dataset_import_job_response = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)
print(f"\n\nThe Dataset Import Job with ARN ds_import_job_arn is now {describe_dataset_import_job_response['Status']}.")


### Creating the first Predictor<a class="anchor" id="firstPredictor"></a>

Forecast horizon is the number of number of time points to predicted in the future. Our example is hourly data, we try forecast the next 24 hours, so we can set to 24.

In [ ]:
firstPredictorName = project+'_firstPred'
forecastHorizon = 24

In [ ]:
first_predictor_response=forecast.create_auto_predictor(PredictorName=firstPredictorName, 
                                                        ForecastHorizon=forecastHorizon,
                                                        ForecastFrequency="H", 
                                                        DataConfig= {"DatasetGroupArn": datasetGroupArn})

In [ ]:
firstPredictorArn=first_predictor_response['PredictorArn']
print("Predictor created from CreateAutoPredictor API", firstPredictorArn)

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 40 mins to more than one hour to be **ACTIVE**.

In [ ]:
start_time = time.time()
print(f"Waiting for Predictor with ARN **{firstPredictorArn}** to become **ACTIVE**. This process could take  minutes. Depending on data size and predictor setting，it can take several hours to be **ACTIVE**.\n\nCurrent Status:")

status = util.wait(lambda: forecast.describe_auto_predictor(PredictorArn=firstPredictorArn))

describe_first_auto_predictor_response = forecast.describe_auto_predictor(PredictorArn=firstPredictorArn)
print(f"\n\nThe Predictor with ARN **{firstPredictorArn}** is now **{describe_first_auto_predictor_response['Status']}**.")

time_to_create_first_predictor = (time.time() - start_time) / 60

In [ ]:
print("Predictor details\n")
forecast.describe_auto_predictor(PredictorArn=firstPredictorArn)

#### Get Error Metrics

In [ ]:
forecast.get_accuracy_metrics(PredictorArn=firstPredictorArn)['PredictorEvaluationResults'][0]['TestWindows']

## Create Retraining Predictor<a class="anchor" id="retrainAuto"></a>

Let's assume that 1 day has passed and we have new data. Since we want to ensure that our predictor is trained on the latest data we can now choose the option to re-train our Predictor that we just created above.

We do retraining as we have seen new data and we want to keep our Predictor upto date with the latest data and learn new trends.

As part of the below step, we reimport the data. Note that key_retrain_predictor has one more day of data so we will train the predictor with the latest data available.

In [ ]:
key_retrain_predictor="elec_data/item-demand-time_new.csv"
s3.upload_file(Filename="data/elec_data_small_new.csv", Bucket=bucketName, Key=key_retrain_predictor)
s3DataPath = "s3://"+bucketName+"/"+key_retrain_predictor

datasetImportJobName = 'Electricity_New_Import_TTS'
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )





In [ ]:

ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print("DatasetImportJob", ds_import_job_arn)

In [ ]:
print("Checking status for DatasetImport Job: ", ds_import_job_arn)

status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn))

describe_dataset_import_job_response = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)
print(f"\n\nThe Dataset Import Job with ARN {ds_import_job_arn} is now {describe_dataset_import_job_response['Status']}.")


In [ ]:
retrainedPredictorName=project+'_retrainedPred'

For Upgrading the predictor, we pass the referencePredictor Arn an an input to CreateAutoPredictor API
Here referencePredictorArn will be the predictorArn created above.

In [ ]:
referencePredictorArn=firstPredictorArn

print("ReferencePredictorArn: ", referencePredictorArn)

create_auto_predictor_response=forecast.create_auto_predictor(PredictorName=retrainedPredictorName, 
                                                              ReferencePredictorArn=referencePredictorArn)

In [ ]:
retrainingPredictorArn=create_auto_predictor_response['PredictorArn']

In [ ]:
print("New Retrained Predictor Arn: ", retrainingPredictorArn)

Now we wait for the new retrained AutoPredictor to complete and it can take 1 hours.

In [ ]:
start_time = time.time()
print(f"Waiting for Predictor with ARN **{retrainingPredictorArn}** to become **ACTIVE**. This process could take 60 minutes. Depending on data size and predictor setting，it can take several hours to be **ACTIVE**.\n\nCurrent Status:")

status = util.wait(lambda: forecast.describe_auto_predictor(PredictorArn=retrainingPredictorArn))

describe_auto_predictor_response = forecast.describe_auto_predictor(PredictorArn=retrainingPredictorArn)
print(f"\n\nThe Predictor with ARN **{retrainingPredictorArn}** is now **{describe_auto_predictor_response['Status']}**.")
time_to_retrain = (time.time() - start_time) / 60

Here we describe the newly created retrained predictor for comparing against the existing predictor

In [ ]:
forecast.describe_auto_predictor(PredictorArn=retrainingPredictorArn)

In [ ]:
forecast.get_accuracy_metrics(PredictorArn=retrainingPredictorArn)

Here we publish metrics to denote the time taken to create the first predictor and retraining the predictor

In [ ]:
print(f"Time to create first predictor {time_to_create_first_predictor} mins")
print(f"Time to retrain the predictor {time_to_retrain} mins")

# Cleanup<a class="anchor" id="cleanup"></a>

Once we have completed the above steps, we can start to cleanup the resources we created. All the created resources can be deleted using `delete_resource_tree` and it is an asynchronous operation, so we have added the helpful `wait_till_delete` function. To learn more about deleting a parent resource and all its child resources, visit [DeleteResourceTree](https://docs.aws.amazon.com/forecast/latest/dg/API_DeleteResourceTree.html) API. 
Resource Limits documented <a href="https://docs.aws.amazon.com/forecast/latest/dg/limits.html">here</a>.

Note: Below lines are commented to ensure resource deletion does not happen by mistake. Please un-comment below to delete the resources.

In [ ]:
# Delete the DatasetGroup, and all its child resources such as Predictor, PredictorBacktestExportJob, Forecast and ForecastExportJob
#util.wait_till_delete(lambda: forecast.delete_resource_tree(ResourceArn=datasetGroupArn))

In [ ]:
# Delete the Dataset and its child DatasetImportJob resources:
#util.wait_till_delete(lambda: forecast.delete_resource_tree(ResourceArn=datasetArn))

In [ ]:
# Delete IAM role
#util.delete_iam_role( role_name )